In [1]:
! pip install findspark


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
! pip install elasticsearch==7.9.0


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
import findspark
from elasticsearch import Elasticsearch, helpers
import time

In [4]:
findspark.init("/opt/spark")

In [5]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import col, struct

In [6]:
! wget https://github.com/erkansirin78/datasets/raw/master/market1mil.csv.gz

--2024-06-12 12:18:57--  https://github.com/erkansirin78/datasets/raw/master/market1mil.csv.gz
Resolving github.com (github.com)... failed: Temporary failure in name resolution.
wget: unable to resolve host address ‘github.com’


In [7]:
spark = (
    SparkSession.builder
    .appName("Spark Elasticsearch")
    .master("local[2]")
    .config("spark.driver.memory","2048m")
    .config("spark.sql.shuffle.partitions", 4)
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    .config("spark.jars.packages", "org.elasticsearch:elasticsearch-spark-30_2.12:7.12.1,commons-httpclient:commons-httpclient:3.1,io.delta:delta-core_2.12:2.4.0") 
    .getOrCreate()
)

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.elasticsearch#elasticsearch-spark-30_2.12 added as a dependency
commons-httpclient#commons-httpclient added as a dependency
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-580bb3e0-ae08-4d4a-b04e-fc59fc50f292;1.0
	confs: [default]
	found org.elasticsearch#elasticsearch-spark-30_2.12;7.12.1 in central
	found org.scala-lang#scala-reflect;2.12.8 in central
	found org.slf4j#slf4j-api;1.7.6 in central
	found commons-logging#commons-logging;1.1.1 in central
	found javax.xml.bind#jaxb-api;2.3.1 in central
	found com.google.protobuf#protobuf-java;2.5.0 in central
	found org.apache.spark#spark-yarn_2.12;3.0.1 in central
	found commons-httpclient#commons-httpclient;3.1 in central
	found commons-codec#commons-codec;1.2 in central
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found o

In [8]:
!gzip -d market1mil.csv.gz

gzip: market1mil.csv already exists; do you wish to overwrite (y or n)? ^C


In [9]:
! pwd

/


In [10]:
df=spark.read.format("csv") \
.option("header",True) \
.option("sep", ";") \
.option("inferSchema",True) \
.load("file:///market1mil.csv")

In [11]:
df.count()

999853

In [12]:
df1=df.dropna()

In [13]:
df1.count()

24/06/12 12:21:06 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


597075

In [14]:
df1.limit(5).toPandas()

,LOGICALREF,COUNT_,ITEMCODE,ITEMNAME,FICHENO,DATE_,AMOUNT,PRICE,LINENETTOTAL,LINENET,...,CLIENTNAME,BRANDCODE,BRAND,CATEGORY_NAME1,CATEGORY_NAME2,CATEGORY_NAME3,STARTDATE,ENDDATE,SPECODE,CAPIBLOCK_CREADEDDATE
0,2,1,20868,KIRMIZI MERCIMEK,15560,2.01.2017 00:00,"1,006","2,8","2,82","2,79",...,Hanım CANBULAT,167,BAKLİYAT,GIDA,BAKLİYAT,AÇIK BAKLİYAT,3.01.2017 09:25,3.01.2017 09:25,K,14.07.2018 01:50
1,3,1,8583,"TEST MATIK 1,5 KG NORMAL",15560,2.01.2017 00:00,1,"4,95","4,95","4,19",...,Hanım CANBULAT,229,TEST,DETERJAN TEMİZLİK,ÇAMAŞIR YIKAMA,TOZ DETERJAN,3.01.2017 09:25,3.01.2017 09:25,K,14.07.2018 01:50
2,4,1,1454,BIZIM MAKARNA BONCUK,15560,2.01.2017 00:00,1,"1,1","1,1","1,02",...,Hanım CANBULAT,146,ÜLKER,GIDA,MAKARNA,MAKARNA,3.01.2017 09:25,3.01.2017 09:25,K,14.07.2018 01:50
3,5,1,13519,FILIZ MAKARNA KISA KESME 500 GR,15560,2.01.2017 00:00,1,"1,1","1,1","1,02",...,Hanım CANBULAT,52,FİLİZ,GIDA,MAKARNA,MAKARNA,3.01.2017 09:25,3.01.2017 09:25,K,14.07.2018 01:50
4,6,1,8639,BINGO ULTRA CAM.SUYU 750 ML KLASIK,15560,2.01.2017 00:00,1,"2,45","2,45","2,08",...,Hanım CANBULAT,224,BİNGO,DETERJAN TEMİZLİK,ÇAMAŞIR YIKAMA,ÇAMAŞIR SULARI,3.01.2017 09:25,3.01.2017 09:25,K,14.07.2018 01:50


In [15]:
df1.printSchema()

root
 |-- LOGICALREF: string (nullable = true)
 |-- COUNT_: integer (nullable = true)
 |-- ITEMCODE: integer (nullable = true)
 |-- ITEMNAME: string (nullable = true)
 |-- FICHENO: integer (nullable = true)
 |-- DATE_: string (nullable = true)
 |-- AMOUNT: string (nullable = true)
 |-- PRICE: string (nullable = true)
 |-- LINENETTOTAL: string (nullable = true)
 |-- LINENET: string (nullable = true)
 |-- BRANCHNR: integer (nullable = true)
 |-- BRANCH: string (nullable = true)
 |-- SALESMAN: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- REGION: string (nullable = true)
 |-- LATITUDE: string (nullable = true)
 |-- LONGITUDE: string (nullable = true)
 |-- CLIENTCODE: string (nullable = true)
 |-- CLIENTNAME: string (nullable = true)
 |-- BRANDCODE: string (nullable = true)
 |-- BRAND: string (nullable = true)
 |-- CATEGORY_NAME1: string (nullable = true)
 |-- CATEGORY_NAME2: string (nullable = true)
 |-- CATEGORY_NAME3: string (nullable = true)
 |-- STARTDATE: string (

In [16]:
df2 = (df1.select(
    col("LOGICALREF").cast("integer"),
    col("ITEMCODE").cast("integer"),
    col("ITEMNAME").cast("string"),
    col("AMOUNT").cast("string"),
    col("PRICE").cast("string"),
    col("LINENETTOTAL").cast("string"),
    col("BRANCH").cast("string"),  
    col("CITY").cast("string"),    
    col("BRAND").cast("string"),   
    col("STARTDATE").cast("string")   
))

In [17]:
df2.limit(5).toPandas()

,LOGICALREF,ITEMCODE,ITEMNAME,AMOUNT,PRICE,LINENETTOTAL,BRANCH,CITY,BRAND,STARTDATE
0,2,20868,KIRMIZI MERCIMEK,"1,006","2,8","2,82",Batman Subesi,Batman,BAKLİYAT,3.01.2017 09:25
1,3,8583,"TEST MATIK 1,5 KG NORMAL",1,"4,95","4,95",Batman Subesi,Batman,TEST,3.01.2017 09:25
2,4,1454,BIZIM MAKARNA BONCUK,1,"1,1","1,1",Batman Subesi,Batman,ÜLKER,3.01.2017 09:25
3,5,13519,FILIZ MAKARNA KISA KESME 500 GR,1,"1,1","1,1",Batman Subesi,Batman,FİLİZ,3.01.2017 09:25
4,6,8639,BINGO ULTRA CAM.SUYU 750 ML KLASIK,1,"2,45","2,45",Batman Subesi,Batman,BİNGO,3.01.2017 09:25


In [18]:
df2.count()

597075

In [19]:
df2.printSchema()

root
 |-- LOGICALREF: integer (nullable = true)
 |-- ITEMCODE: integer (nullable = true)
 |-- ITEMNAME: string (nullable = true)
 |-- AMOUNT: string (nullable = true)
 |-- PRICE: string (nullable = true)
 |-- LINENETTOTAL: string (nullable = true)
 |-- BRANCH: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- BRAND: string (nullable = true)
 |-- STARTDATE: string (nullable = true)



In [43]:
df3 = (df2
    .withColumn("AMOUNT", F.regexp_replace("AMOUNT", ",", ".").cast(FloatType()))
    .withColumn("PRICE", F.regexp_replace("PRICE", ",", ".").cast(FloatType()))
    .withColumn("LINENETTOTAL", F.regexp_replace("LINENETTOTAL", ",", ".").cast(FloatType()))
    .withColumn(
        "STARTDATE",
        F.unix_timestamp(F.to_timestamp(F.col("STARTDATE"), "d.MM.yyyy HH:mm"))
    )
)


In [44]:
df3.limit(5).toPandas()

,LOGICALREF,ITEMCODE,ITEMNAME,AMOUNT,PRICE,LINENETTOTAL,BRANCH,CITY,BRAND,STARTDATE
0,2,20868,KIRMIZI MERCIMEK,1.006,2.80,2.82,Batman Subesi,Batman,BAKLİYAT,1483435500
1,3,8583,"TEST MATIK 1,5 KG NORMAL",1.000,4.95,4.95,Batman Subesi,Batman,TEST,1483435500
2,4,1454,BIZIM MAKARNA BONCUK,1.000,1.10,1.10,Batman Subesi,Batman,ÜLKER,1483435500
3,5,13519,FILIZ MAKARNA KISA KESME 500 GR,1.000,1.10,1.10,Batman Subesi,Batman,FİLİZ,1483435500
4,6,8639,BINGO ULTRA CAM.SUYU 750 ML KLASIK,1.000,2.45,2.45,Batman Subesi,Batman,BİNGO,1483435500


In [45]:
df3.printSchema()

root
 |-- LOGICALREF: integer (nullable = true)
 |-- ITEMCODE: integer (nullable = true)
 |-- ITEMNAME: string (nullable = true)
 |-- AMOUNT: float (nullable = true)
 |-- PRICE: float (nullable = true)
 |-- LINENETTOTAL: float (nullable = true)
 |-- BRANCH: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- BRAND: string (nullable = true)
 |-- STARTDATE: long (nullable = true)



In [46]:
odev_index =  {
  "settings": {
    "index": {
      "analysis": {
        "analyzer": {
          "custom_analyzer":
          {
            "type":"custom",
            "tokenizer":"standard",
            "filter":[
              "lowercase", "custom_edge_ngram"
            ]
          }
        },
        "filter": {
          "custom_edge_ngram": {
            "type": "edge_ngram",
            "min_gram":2,
            "max_gram": 10
            }
          }
        }
      }
    },
    "mappings": {
    "properties": {
      "LOGICALREF":  {"type": "integer"},  
      "ITEMCODE":    {"type": "integer" }, 
      "ITEMNAME":    {"type": "text"},
      "AMOUNT":      {"type": "float"},
      "PRICE":       {"type": "float"},
      "LINENETTOTAL":{"type": "float"},
      "BRANCH":      {"type": "keyword"},
      "CITY":        {"type": "keyword"},
      "BRAND":       {"type": "keyword"},
      "STARTDATE":   {"type": "date"},
    }
  }
  }

In [47]:
es = Elasticsearch("http://elasticsearch:9200")

In [48]:
try:
    es.indices.delete("odev_index")
    print("odev_index silindi.")
except:
    print("odev_index yok")

odev_index silindi.


In [49]:
es.indices.create("odev_index", body=odev_index)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'odev_index'}

In [50]:
start_time = time.time()
df3.write \
    .format("org.elasticsearch.spark.sql") \
    .mode("append") \
    .option("es.nodes", "es") \
    .option("es.port","9200") \
    .save("odev_index")
print("----- %s secs -----" %(time.time() - start_time))

----- 62.700568437576294 secs -----
